## Convenient environment for quantum annealer
### Install pyQUBO from Recruit Communications Co. Ltd.
    pip install pyqubo
### Install openJij from Jij Inc.  (startup from Tohoku University)
    pip install -U cmake (in google collaboratory, update cmake)
    pip install open jij

# Solve number partition problem

### import pyQUBO

In [282]:
from pyqubo  import Array

Array is a class 

### Prepare spin variables

In [292]:
N = 8
vartype = "SPIN"
x = Array.create("x",shape=N,vartype=vartype)

"x" is name of variables  
shape specifies the shape of variables as vector, matrix, or...  
vartype selects -1 or 1 by "SPIN" and 0 or 1by "BINARY"

In [293]:
print(x)

Array([Spin(x[0]), Spin(x[1]), Spin(x[2]), Spin(x[3]), Spin(x[4]), Spin(x[5]), Spin(x[6]), Spin(x[7])])


### Let generate integers

In [294]:
import numpy as np
n = np.linspace(1,N,N)

In [295]:
print(n)

[1. 2. 3. 4. 5. 6. 7. 8.]


### Define cost function

In [296]:
E_cost = (np.sum(n*x))**2

### Compile the cost function

In [297]:
model = E_cost.compile()

### Get qubo matrix

In [308]:
h,J, offset = model.to_ising()

### import openJij

In [309]:
import openjij as jij

### Prepare simulation of quantum annealing

In [310]:
sampler = jij.SQASampler(beta=10.0, gamma=1.0, trotter=4, step_length=10, step_num=100, schedule=None, iteration=1)

This is done by quantum Monte-Carlo simulation  
gamma = strength of quantum fluctuation  
iteration = number of reads  
step_num = number of MCS  
trotter = Trotter number  
step_length = length of MCS in the same gamma   

### Let's simulate quantum annealing

In [311]:
response = sampler.sample_ising(h,J)

### show results

In [312]:
print(response)

iteration : 1, minimum energy : -204.0, var_type : SPIN
indices: ['x[5]', 'x[2]', 'x[3]', 'x[6]', 'x[7]', 'x[1]', 'x[4]', 'x[0]'] 
minmum energy state sample : [1, 1, -1, 1, -1, 1, -1, -1]


### minimum sample

In [313]:
response.samples[0]

{'x[5]': 1,
 'x[2]': 1,
 'x[3]': -1,
 'x[6]': 1,
 'x[7]': -1,
 'x[1]': 1,
 'x[4]': -1,
 'x[0]': -1}

### decode solution through openJij and pyQUBO

In [314]:
def decode_spin(sol,vartype,feed_dict = None):
    dsol, broken, energy = model.decode_solution(sol, feed_dict = feed_dict, vartype=vartype)
    spin = np.zeros(N)
    for keys in dsol["x"]:
        spin[keys] = dsol["x"][keys]
    return spin,energy 

We define an original function for decoding the solution from openJij with pyQUBO into a standard expression.  
reposense.min["min_state"] is written on the indices in openJij.
Here we arrange the original order of the variables by our function.

In [315]:
spin,energy = decode_spin(response.samples[0],vartype='SPIN')

In [316]:
print(energy,spin)

0.0 [-1.  1.  1. -1. -1.  1.  1. -1.]
